<a href="https://colab.research.google.com/github/saga3121/House/blob/main/House_Price_Sagar_Kadam_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the data

In [4]:
df= pd.read_csv('/content/drive/My Drive/House.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import metrics
from sklearn import preprocessing

In [5]:
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

(   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
 0   1          60       RL         65.0     8450   Pave   NaN      Reg   
 1   2          20       RL         80.0     9600   Pave   NaN      Reg   
 2   3          60       RL         68.0    11250   Pave   NaN      IR1   
 3   4          70       RL         60.0     9550   Pave   NaN      IR1   
 4   5          60       RL         84.0    14260   Pave   NaN      IR1   
 
   LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
 0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
 1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
 2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
 3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
 4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   
 
   YrSold  SaleType  SaleCondition  SalePrice  
 0   2

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import numpy as np

# Drop columns with too many missing values
df_cleaned = df.drop(columns=['Alley', 'PoolQC', 'Fence', 'MiscFeature'])

# Fill missing values for numerical columns with median
imputer_num = SimpleImputer(strategy='median')
df_cleaned[df_cleaned.select_dtypes(include=[np.number]).columns] = imputer_num.fit_transform(df_cleaned.select_dtypes(include=[np.number]))

# Fill missing values for categorical columns with the most frequent value
imputer_cat = SimpleImputer(strategy='most_frequent')
df_cleaned[df_cleaned.select_dtypes(include=[object]).columns] = imputer_cat.fit_transform(df_cleaned.select_dtypes(include=[object]))

# Encode categorical variables
label_encoders = {}
for column in df_cleaned.select_dtypes(include=[object]).columns:
    le = LabelEncoder()
    df_cleaned[column] = le.fit_transform(df_cleaned[column])
    label_encoders[column] = le

# Split the data into features and target variable
X = df_cleaned.drop(columns=['SalePrice', 'Id'])
y = df_cleaned['SalePrice']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate the model on the test set
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

# Get the coefficients of the features
feature_importance = pd.Series(model.coef_, index=X.columns).sort_values(ascending=False)

r2, feature_importance.head(10)  # Top 10 most significant features


(0.8467344786431392,
 Street          19236.504260
 LandSlope       13082.551351
 OverallQual     11461.437088
 GarageCars      11208.796147
 BsmtFullBath     8772.952722
 Fireplaces       6416.614099
 Functional       4574.613777
 OverallCond      4227.351229
 RoofMatl         4170.594613
 BsmtCond         3859.673444
 dtype: float64)

The top 10 most significant features in predicting house prices, based on their coefficients, are:

Street:
17879.72
17879.72
LandSlope:
12826.80
12826.80
GarageCars:
11516.42
11516.42
OverallQual:
11491.74
11491.74
BsmtFullBath:
8793.12
8793.12
Fireplaces:
6275.79
6275.79
Functional:
4651.01
4651.01
OverallCond:
4276.54
4276.54
RoofMatl:
4076.65
4076.65
MasVnrType:
3941.60
3941.60